In [ ]:
import pandas as pd
import numpy as np

# Таблица 1
(получена из других ноутбуков)
Сравниваем с фоном

In [ ]:
table1 = np.array([['Exons',14,	'8.3%',	800,	'8.4%',	3,	'30%'],
       ['Introns',10,	'6%',	965,	'10.1%',	0,	'0%'],
       ['Promoters (1000 up from TSS)', 34,	'20.2%',	4836,	'50.8%',	0,	'0%'],
       ['Downstream (200 bp)',17,	'10.1%',	377,	'4%',	1,	'10%'],
       ['Intergenic', 115,	'68.5%',	7868,	'82.6%',	7,	'70%']],
      dtype=object)
df1 = pd.DataFrame(table1)
df1.columns  = ['Участок','Число квадруплексов','Доля квадруплексов','Число предсказаний Zhun','Доля предсказаний Zhun','Число предсказаний ZDNABERT','Доля предсказаний ZDNABERT']

In [ ]:
df1

,Участок,Число квадруплексов,Доля квадруплексов,Число предсказаний Zhun,Доля предсказаний Zhun,Число предсказаний ZDNABERT,Доля предсказаний ZDNABERT
0,Exons,14,8.3%,800,8.4%,3,30%
1,Introns,10,6%,965,10.1%,0,0%
2,Promoters (1000 up from TSS),34,20.2%,4836,50.8%,0,0%
3,Downstream (200 bp),17,10.1%,377,4%,1,10%
4,Intergenic,115,68.5%,7868,82.6%,7,70%


In [ ]:
%%bash
for f in exons introns promoters downstream intergenic; do
  echo -n "$f length: "
  awk '{sum += $3 - $2} END {print sum}' ${f}.bed
done

exons length: 18553834
introns length: 3280720
promoters length: 11966124
downstream length: 2455713
intergenic length: 11133685


In [ ]:
lengths = {
    "Exons": 18553834,
    "Introns": 3280720,
    "Promoters (1000 up from TSS)": 11966124,
    "Downstream (200 bp)": 2455713,
    "Intergenic": 11133685
}

background = pd.DataFrame({
    "Участок": list(lengths.keys()),
    "Длина": list(lengths.values())
})

In [ ]:
%%bash
for f in exons introns promoters downstream intergenic; do
  echo -n "quadruplex length in $f: "
  awk '{sum += $3 - $2} END {print sum}' quadr_in_${f}.bed
done

quadruplex length in exons: 419
quadruplex length in introns: 251
quadruplex length in promoters: 833
quadruplex length in downstream: 347
quadruplex length in intergenic: 2792


In [ ]:
q_lengths = {
    "Exons": 419,
    "Introns": 251,
    "Promoters (1000 up from TSS)": 833,
    "Downstream (200 bp)": 347,
    "Intergenic": 2792
}

background["Длины квадруплексов"] = list(q_lengths.values())

In [ ]:
background["Доля квадруплексов по длине"] = \
 ((background["Длины квадруплексов"] / background["Длина"] * 100).round(2))

In [ ]:
%%bash
for f in exons introns promoters downstream intergenic; do
  echo -n "z-dna length in $f: "
  awk '{sum += $3 - $2} END {print sum}' zdna_in_${f}.bed
done

z-dna length in exons: 14880
z-dna length in introns: 19072
z-dna length in promoters: 115254
z-dna length in downstream: 7674
z-dna length in intergenic: 158764


In [ ]:
z_lengths = {
    "Exons": 14880,
    "Introns": 19072,
    "Promoters (1000 up from TSS)": 115254,
    "Downstream (200 bp)": 7674,
    "Intergenic": 158764
}

background["Длины Z-ДНК"] = list(z_lengths.values())

In [ ]:
background["Доля Z-ДНК по длине"] = \
 (background["Длины Z-ДНК"] / background["Длина"] * 100).round(2)

In [ ]:
%%bash
for f in exons downstream intergenic; do
  echo -n "zdnabert length in $f: "
  awk '{sum += $3 - $2} END {print sum}' ZDNABERT_in_${f}.bed
done

zdnabert length in exons: 48
zdnabert length in downstream: 13
zdnabert length in intergenic: 107


In [ ]:
bert_lengths = {
    "Exons": 48,
    "Introns": 0,
    "Promoters (1000 up from TSS)": 0,
    "Downstream (200 bp)": 13,
    "Intergenic": 107
}

background["Длины ZDNABERT"] = list(bert_lengths.values())

In [ ]:
background["Доля предсказаний ZDNAbert по длине"] = \
 (background["Длины ZDNABERT"] / background["Длина"] * 100)

In [ ]:
merged = pd.merge(df1, background[[
    "Участок",
    "Доля квадруплексов по длине",
    "Доля Z-ДНК по длине",
    "Доля предсказаний ZDNAbert по длине",
]], on="Участок")

In [ ]:
merged

,Участок,Число квадруплексов,Доля квадруплексов,Число предсказаний Zhun,Доля предсказаний Zhun,Число предсказаний ZDNABERT,Доля предсказаний ZDNABERT,Доля квадруплексов по длине,Доля Z-ДНК по длине,Доля предсказаний ZDNAbert по длине
0,Exons,14,8.3%,800,8.4%,3,30%,0.00,0.08,0.000259
1,Introns,10,6%,965,10.1%,0,0%,0.01,0.58,0.000000
2,Promoters (1000 up from TSS),34,20.2%,4836,50.8%,0,0%,0.01,0.96,0.000000
3,Downstream (200 bp),17,10.1%,377,4%,1,10%,0.01,0.31,0.000529
4,Intergenic,115,68.5%,7868,82.6%,7,70%,0.03,1.43,0.000961
